In [1]:
!pip install llama-index


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import textwrap

import pandas as pd
import phoenix as px
from langchain import OpenAI
from llama_index import StorageContext, load_index_from_storage
from llama_index.response.schema import Response

/Users/xandersong/.virtualenvs/phoenixdev/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/xandersong/.virtualenvs/phoenixdev/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/xandersong/.virtualenvs/phoenixdev/lib/python3.8/si

In [3]:
# database_df = pd.read_parquet("/Users/xandersong/Desktop/llama-index-data-full/splits/database.parquet")
# query_df = pd.read_parquet("/Users/xandersong/Desktop/llama-index-data/splits/query.parquet")
database_df = pd.read_parquet("/Users/xandersong/Downloads/database_openai.parquet")
query_df = pd.read_parquet("/Users/xandersong/Downloads/query_openai.parquet")

In [4]:
database_df.head()

,article_index,granular_subject,broad_subject,text,text_vector
0,139,FC_Barcelona,Sports,"Two days later, it was announced that Luis Enr...","[0.0009573115967214108, 0.0031192197930067778,..."
1,250,"New_Haven,_Connecticut",Geography and Places,A second New Haven gastronomical claim to fame...,"[-0.0028259255923330784, 0.01535571739077568, ..."
2,306,Mandolin,Music,"There are many styles of mandolin, but four ar...","[-0.005895992740988731, 0.015205282717943192, ..."
3,183,Chicago_Cubs,Sports,The Chicago Cubs are an American professional ...,"[-0.0005755521706305444, -0.015405493788421154..."
4,430,Saint_Barth%C3%A9lemy,Geography and Places,Located approximately 250 kilometres (160 mi) ...,"[0.0016602941323071718, -0.009640098549425602,..."


In [5]:
query_df.head()

,id,granular_subject,broad_subject,paragraph_index,text,is_answerable,text_vector
0,5729499d1d04691400779256,"New_Haven,_Connecticut",Geography and Places,6,Exactly who were the slaves upon the Spanish s...,True,"[-0.008130733855068684, -0.017305415123701096,..."
1,5a555aad134fea001a0e1a4b,Computer_security,Science and Technology,15,How wide-scale were the losses?,False,"[-0.007133032660931349, -0.02135872282087803, ..."
2,5a2d6060f28ef0001a52650e,"Santa_Monica,_California",Geography and Places,16,On what street does the Santa Monica Freeway b...,False,"[0.025623859837651253, -0.007313745096325874, ..."
3,5730245ca23a5019007fce41,Windows_8,Science and Technology,9,What is the name of the character Microsoft us...,True,"[-0.016982482746243477, -0.0013794071273878217..."
4,5ace3e9b32bba1001ae4a036,"Ann_Arbor,_Michigan",Geography and Places,19,What week in March is the Ann Arbor Art Fairs ...,False,"[-0.006391833536326885, -0.02348998934030533, ..."


In [6]:
query_df[(query_df["is_answerable"] == "True")].head()["text"].to_list()

['Exactly who were the slaves upon the Spanish slaveship?',
 'What is the name of the character Microsoft used to make Windows 8 seem more personable?',
 'Approximately how much was spent on non-food retail sales in London in 2010?',
 "What type of campus did Northwestern begin construction of during Walter Dill Scott's presidency?",
 'What did the Shanghai Stock Exchange do?']

In [ ]:
query_df[(query_df["is_answerable"] == "False")].head()["text"].to_list()

In [7]:
storage_context = StorageContext.from_defaults(
    persist_dir="/Users/xandersong/Desktop/llama-index-data-full/indexes/database_index"
)
# model_name = "text-davinci-003"
model_name = "gpt-4"
llm = OpenAI(temperature=0, model_name=model_name)
index = load_index_from_storage(storage_context, llm=llm)
query_engine = index.as_query_engine()

/Users/xandersong/.virtualenvs/phoenixdev/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/xandersong/.virtualenvs/phoenixdev/lib/python3.8/site-packages/langchain/llms/openai.py:687: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [8]:
def display_llama_index_response(response: Response) -> None:
    """
    Displays a LlamaIndex response and its source nodes.
    """

    print("Response")
    print("========")
    for line in textwrap.wrap(response.response.strip(), width=80):
        print(line)
    print()

    print("Source Nodes")
    print("============")
    print()

    for source_node in response.source_nodes:
        print(f"doc_id: {source_node.node.doc_id}")
        print(f"score: {source_node.score}")
        print()
        for line in textwrap.wrap(source_node.node.text, width=80):
            print(line)
        print()

In [ ]:
query = 'What is the name of the character Microsoft used to make Windows 8 seem more personable?'
# query = 'On what street does the Santa Monica Freeway begin?'
response = query_engine.query(query)
display_llama_index_response(response)

In [ ]:
database_df_ = database_df.sample(frac=1)
database_df_["is_answerable"] = "unknown"
query_df_ = query_df.sample(frac=1)
query_df_["is_answerable"] = query_df_["is_answerable"].astype(str)

In [ ]:
schema = px.Schema(
    embedding_feature_column_names={
        "text_embedding": px.EmbeddingColumnNames(
            vector_column_name="text_vector",
            raw_data_column_name="text",
          )
    },
    tag_column_names=["granular_subject", "broad_subject", "is_answerable"],
)

In [ ]:
database_ds = px.Dataset(database_df_, schema, name="database")
query_ds = px.Dataset(query_df_, schema, name="query")

In [ ]:
px.launch_app(primary=query_ds, reference=database_ds)

In [ ]:
database_df_ = database_df[~database_df["granular_subject"].isin(["Beyoncé"])]

In [ ]:
import random

random_subjects = random.sample(database_df["granular_subject"].unique().tolist(), 10)
random_subjects

In [ ]:
database_df_ = database_df[database_df["granular_subject"].isin(random_subjects)]
query_df_ = query_df[query_df["granular_subject"].isin(random_subjects)]

In [ ]:
query_df_ = query_df.groupby(["granular_subject", "paragraph_index"], as_index=False).first().reset_index(drop=True)

In [ ]:
len(query_df.sample(n=2500))

In [ ]:
len(database_df)

In [ ]:
import pandas as pd


def sample_by_percentage(df, percentage_map):
    def sample_group(group):
        subject = group.name
        frac = percentage_map.get(subject, 0)
        return group.sample(frac=frac)

    sampled_df = df.groupby("broad_subject").apply(sample_group).reset_index(drop=True)
    return sampled_df


# Example usage
data = {
    "broad_subject": ["Math", "Science", "Math", "Science", "Math", "Science"],
    "score": [90, 85, 95, 88, 92, 80],
}
df = pd.DataFrame(data)

# Define the percentage map
percentage_map = {
    "Math": 0.5,  # Sample 50% of rows for Math
    "Science": 0.3,  # Sample 30% of rows for Science
}

# Apply the function to the DataFrame
sampled_df = sample_by_percentage(df, percentage_map)
print(sampled_df)

In [ ]:
broad_subject_to_sample_percentage = {
    "Architecture": 5.0,
    "Business and Economy": 0.5,
    "Education": 0.5,
    "Entertainment and Arts": 0.5,
    "Geography and Places": 0.5,
    "Health and Medicine": 0.5,
    "History": 0.5,
    "Language and Linguistics": 0.5,
    "Law and Legal": 0.5,
    "Literature": 0.5,
    "Media and Communication": 0.5,
    "Music": 0.5,
    "Nature and Environment": 0.5,
    "Nonprofit Organizations": 0.5,
    "People and Ethnicity": 0.5,
    "Philosophy": 0.5,
    "Politics and Government": 0.5,
    "Religion and Spirituality": 0.5,
    "Science and Technology": 0.5,
    "Social Sciences": 0.5,
    "Sports": 0.5,
}

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken

def get_token_count(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(text)
    return len(tokens)

database_df["text"].map(get_token_count).sort_values(ascending=False).head(10)